# Week 7 Wednesday


## Announcements

* Swap Thursday discussion and Friday lecture
* HW6 due Wednesday
* HW7 has been posted

## Plan
* Multi-class Classification

In [ ]:
import pandas as pd
import numpy as np
import altair as alt
import seaborn as sns
df = sns.load_dataset("penguins").dropna(axis=0)

## Recap of Monday

On Monday we used Logistic Regression with the flipper length and bill length columns in the penguins dataset to predict if a penguin is in the Chinstrap species.

In [ ]:
df = sns.load_dataset("penguins").dropna(axis=0).copy()
df["isChinstrap"] = (df["species"] == "Chinstrap")
cols = ["flipper_length_mm", "bill_length_mm"]

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(df[cols], df["isChinstrap"])
df["pred"] = clf.predict(df[cols])

## Multi-class Classification

### Model 
Let $\tilde{x}\in\mathbb{R}^{p+1}$ denotes the augmented row vector (one sample). We approximate the probabilities to take value in $K$ classes as

$$
f(\mathbf{x};W) =
\begin{pmatrix}
P(y = 1 | \mathbf{x}; \mathbf{w}) \\
P(y = 2 | \mathbf{x}; \mathbf{w}) \\
\vdots \\
P(y = K | \mathbf{x}; \mathbf{w})
\end{pmatrix}
=
\frac{1}{ \sum_{k=1}^{K}{\exp\big(\tilde{x}\mathbf{w}_{k}\big) }}
\begin{pmatrix}
\exp(\tilde{x}\mathbf{w}_{1} ) \\
\exp(\tilde{x}\mathbf{w}_{2} ) \\
\vdots \\
\exp(\tilde{x}\mathbf{w}_{K} ) \\
\end{pmatrix}.
$$
where we have $K$ sets of parameters, $\mathbf{w}_1, \mathbf{w}_2, \dots, \mathbf{w}_K$, and the sum factor normalizes the results to be a probability.

$\mathbf{W}$ is an $(p+1)\times K$ matrix containing all $K$ sets of parameters, obtained by concatenating $\mathbf{w}_1, \mathbf{w}_2, \dots, \mathbf{w}_K$ into columns, so that $\mathbf{w}_k = (w_{k0}, \dots, w_{kp})^{\top}\in \mathbb{R}^{p+1}$.

$$
\mathbf{w} = \left(
\begin{array}{cccc}| & | & | & | \\
\mathbf{w}_1 & \mathbf{w}_2 & \cdots & \mathbf{w}_K \\
| & | & | & |
\end{array}\right),
$$
and $\tilde{X}\mathbf{W}$ is valid and useful in vectorized code.

**Another Expression**: Introduce the hidden variable $\mathbf{z} = (z_{1},...,z_{K})$ and define 

$$ \mathbf{z}= \tilde{\mathbf{x}} \mathbf{W}$$
or element-wise written as 
$$z_{k} = \tilde{\mathbf{x}} \mathbf{w_{k}}, k = 1,2,...,K$$

Then the **predicted probability distribution** can be denoted as 

$$f(\mathbf{x};W) = \sigma(z)\in\mathbb{R}^{k}$$
where $\sigma(\mathbf{z})$ is called the [soft-max function](https://en.wikipedia.org/wiki/Softmax_function) which is defined as 

$$ \sigma (\mathbf {z} )_{i}={\frac {e^{z_{i}}}{\sum _{j=1}^{K}e^{z_{j}}}}{\text{ for }}i=1,\dotsc ,K{\text{ and }}\mathbf {z} =(z_{1},\dotsc ,z_{K})\in \mathbb {R} ^{K}$$

This is a valid probability distribution with $K$ classes because you can check its element-wise sum is one and each component is positive.

This can be assumed as the (degenerate) simplest example of neural network that we're going to learn in later lectures, and that's why some people call multi-class logistic regression (also known as **soft-max logistic regression**) as **one-layer neural network**.

----

### Loss function

Define the following indicator function (and again can be derived from MLE):
$$
1_{\{y = k\}} = 1_{\{k\}}(y) = \delta_{yk} = \begin{cases}
1 & \text{when } y = k,
\\[5pt]
0 & \text{otherwise}.
\end{cases}
$$

Loss function is again using the cross entropy:

$$
\begin{aligned}
L (\mathbf{W};X,\mathbf{y})  & = - \frac{1}{N}\sum_{i=1}^N \sum_{k=1}^K
\Bigl\{ 1_{\{y^{(i)} = k\}} \ln P\big(y^{(i)}=k | \mathbf{x}^{(i)} ; \mathbf{w} \big) \Bigr\}
\\
 & = - \frac{1}{N}\sum_{i=1}^N \sum_{k=1}^K
\left\{1_{\{y^{(i)} = k\}} \ln \Bigg( \frac{\exp(\tilde{x}^{(i)}\mathbf{w}_{k})}{\sum_{m=1}^{K} 
\exp\big(\tilde{x}^{(i)}\mathbf{w}_m\big) }  \Bigg)\right\}.
\end{aligned}
$$

Notice that for each term in the summation over N (i.e. fix sample i), only one term is non-zero in the sum of K elements due to the indicator function.


### Gradient descent
After **careful calculation**, the gradient of $L$ with respect the whole $k$-th set of weights is then:

$$
\frac{\partial L }{\partial \mathbf{w}_{k}}
= 
\frac{1}{N}\sum_{i=1}^N 
\left(    \frac{\exp(\tilde{x}^{(i)}\mathbf{w}_{k})} {\sum_{m=1}^{K} 
\exp(\tilde{x}^{(i)}\mathbf{w}_m)} -1_{\{y^{(i)} = k\}} 
\right)\tilde{x}^{(i)}\in\mathbb{R}^{p+1}.
$$

In writing the code, it's helpful to make this as the column vector, and stack all the $K$ gradients together as a new matrix $\mathbf{dW}\in\mathbb{R}^{(p+1)\times K}$. This makes the update of matrix $\mathbf{W}$ very convenient in gradient descent.


### Prediction
The largest estimated probability's class as this sample's predicted label.
$$
\hat{y} = \operatorname{arg}\max_{j} P\big(y = j| \mathbf{x}\big),
$$

* Drop the "pred" column from `df` (we will make a new one below) using the `drop` method and a suitable `axis` keyword argument.

We are using `axis=1` because it is the column labels that are changing (one of the column labels is being removed). 

In [ ]:
df = df.drop("pred", axis = 1)

In [ ]:
df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,isChinstrap
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male,False
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female,False
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female,False
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female,False
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male,False
...,...,...,...,...,...,...,...,...
338,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,Female,False
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female,False
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male,False
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female,False


* Fit a new logistic regression classifier, using the same input features, but this time using the "species" column as our target. 

Even though we have three output classes, the procedure is the same we have been using all along.

In [ ]:
clf = LogisticRegression()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[cols], df["species"], test_size=0.1, random_state=42)

In [ ]:
clf.fit(X_train, y_train)

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

This warning indicates that the algorithm did not converge to a solution within the number of iterations specified by the max_iter parameter.

The most straightforward approach is to increase the max_iter parameter in your logistic regression model. This gives the algorithm more iterations to converge to a solution. However, be aware that setting this number too high might lead to longer training times.

In [ ]:
help(clf)

Help on LogisticRegression in module sklearn.linear_model._logistic object:

class LogisticRegression(sklearn.linear_model._base.LinearClassifierMixin, sklearn.linear_model._base.SparseCoefMixin, sklearn.base.BaseEstimator)
 |  LogisticRegression(penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
 |  
 |  Logistic Regression (aka logit, MaxEnt) classifier.
 |  
 |  In the multiclass case, the training algorithm uses the one-vs-rest (OvR)
 |  scheme if the 'multi_class' option is set to 'ovr', and uses the
 |  cross-entropy loss if the 'multi_class' option is set to 'multinomial'.
 |  (Currently the 'multinomial' option is supported only by the 'lbfgs',
 |  'sag', 'saga' and 'newton-cg' solvers.)
 |  
 |  This class implements regularized logistic regression using the
 |  'liblinear' library, 'newton-cg', '

In [ ]:
clf = LogisticRegression(max_iter=400)
clf.fit(X_train, y_train)

LogisticRegression(max_iter=400)

In [ ]:
clf.score(X_test, y_test)

0.9705882352941176

* Check the `coef_` attribute.  How does it relate to the `coef_` attribute we found above, where we were only considering Chinstrap penguins?

In [ ]:
clf.coef_

array([[-0.10784061, -0.57168209],
       [-0.2536906 ,  0.67608474],
       [ 0.36153121, -0.10440267]])

Remember how the order of `False` and `True` above was important.  Here the order of the penguin species is also important.

In [ ]:
clf.classes_

array(['Adelie', 'Chinstrap', 'Gentoo'], dtype=object)

Here are the predicted probabilities for that "incorrect" row `122` from above. (We know the middle number corresponds to Chinstrap, by looking at the order of the values in `clf.classes_`.)

In [ ]:
clf.predict_proba(df.loc[[122],cols])

array([[9.14508321e-01, 8.54914364e-02, 2.42247434e-07]])

In [ ]:
df.loc[122,:]

species                 Adelie
island               Torgersen
bill_length_mm            40.2
bill_depth_mm             17.0
flipper_length_mm        176.0
body_mass_g             3450.0
sex                     Female
isChinstrap              False
Name: 122, dtype: object

Here is what it looks like if we evaluate the `predict_proba` method on a 4-row sub-DataFrame.  Notice how the output has four rows also and three columns (one column for each target class).

In [ ]:
clf.predict_proba(df.loc[121:124,cols])

array([[9.97326453e-01, 1.66458507e-04, 2.50708838e-03],
       [9.14508321e-01, 8.54914364e-02, 2.42247434e-07],
       [9.07428478e-01, 8.54943022e-03, 8.40220913e-02],
       [9.99954759e-01, 4.24429522e-05, 2.79784347e-06]])

* Add a column "pred" to `df` containing the predicted values.

In [ ]:
df['pred'] = clf.predict(df[cols])

Notice how the "pred" column at the far right contains penguin species strings.  So the `predict` method can output strings, not just numbers or Boolean values.

In [ ]:
df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,isChinstrap,pred
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male,False,Adelie
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female,False,Adelie
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female,False,Adelie
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female,False,Adelie
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male,False,Adelie
...,...,...,...,...,...,...,...,...,...
338,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,Female,False,Gentoo
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female,False,Gentoo
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male,False,Gentoo
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female,False,Gentoo


* Make an Altair scatter plot showing the predicted values.

The portion where the model switches from one prediction to another prediction is called the "decision boundary".  It's hard to recognize the decision boundary in this picture, because there is so much empty space.

In [ ]:
alt.Chart(df).mark_circle().encode(
    x = alt.X(cols[0], scale = alt.Scale(zero=False)),
    y = alt.Y(cols[1], scale = alt.Scale(zero=False)),
    color = 'pred:N'
)

alt.Chart(...)

It is a little hard to see from the above picture how the predictions are made.  It turns out there are a few straight line segments, and on one side of each line segment, one prediction is made, and on the other side, another prediction is made.  We will make a fake dataset from which these "decision boundaries" are more clear.

* Using `np.linspace`, make a NumPy array of 70 equally spaced x-coordinates and 70 equally spaced y-coordinates.  Name these NumPy arrays `x` and `y`.

Notice how the ranges chosen here are chosen so that it matches the approximate ranges of the flipper length and the bill length.

In [ ]:
x = np.linspace(170, 235, 70)
y = np.linspace(30, 60, 70)

In [ ]:
x.shape

(70,)

* Make a DataFrame `df_art` (for "artificial") containing all the possible pairs of coordinates from `x` and `y`.  (We chose `70` above so `df_art` will have `4900` rows, which is a good length for Altair.)

To get these pairs, we can use `itertools.product`.


In [ ]:
from itertools import product

In [ ]:
product(x, y)

In [ ]:
# If we convert it to a list, it's more clear.
list(product(x, y))

[(170.0, 30.0),
 (170.0, 30.434782608695652),
 (170.0, 30.869565217391305),
 (170.0, 31.304347826086957),
 (170.0, 31.73913043478261),
 (170.0, 32.17391304347826),
 (170.0, 32.608695652173914),
 (170.0, 33.04347826086956),
 (170.0, 33.47826086956522),
 (170.0, 33.91304347826087),
 (170.0, 34.34782608695652),
 (170.0, 34.78260869565217),
 (170.0, 35.21739130434783),
 (170.0, 35.65217391304348),
 (170.0, 36.08695652173913),
 (170.0, 36.52173913043478),
 (170.0, 36.95652173913044),
 (170.0, 37.391304347826086),
 (170.0, 37.82608695652174),
 (170.0, 38.26086956521739),
 (170.0, 38.69565217391305),
 (170.0, 39.130434782608695),
 (170.0, 39.565217391304344),
 (170.0, 40.0),
 (170.0, 40.434782608695656),
 (170.0, 40.869565217391305),
 (170.0, 41.30434782608695),
 (170.0, 41.73913043478261),
 (170.0, 42.17391304347826),
 (170.0, 42.608695652173914),
 (170.0, 43.04347826086956),
 (170.0, 43.47826086956522),
 (170.0, 43.91304347826087),
 (170.0, 44.34782608695652),
 (170.0, 44.78260869565217),
 

There are `4900` tuples in this list ($70 \cdot 70$).

In [ ]:
len(list(product(x, y)))

4900

We convert this into a DataFrame with 4900 rows and two columns.

In [ ]:
df_art = pd.DataFrame(list(product(x, y)))
df_art

,0,1
0,170.0,30.000000
1,170.0,30.434783
2,170.0,30.869565
3,170.0,31.304348
4,170.0,31.739130
...,...,...
4895,235.0,58.260870
4896,235.0,58.695652
4897,235.0,59.130435
4898,235.0,59.565217


We give the columns the same names as our input features.

In [ ]:
df_art.columns = cols

In [ ]:
df_art

,flipper_length_mm,bill_length_mm
0,170.0,30.000000
1,170.0,30.434783
2,170.0,30.869565
3,170.0,31.304348
4,170.0,31.739130
...,...,...
4895,235.0,58.260870
4896,235.0,58.695652
4897,235.0,59.130435
4898,235.0,59.565217


* Add a corresponding "pred" column to `df_art`.


We now add a prediction column.  (A warning would be raised if we didn't have the same column names as when we fit the classifier using `clf.fit`.)

In [ ]:
df_art['pred'] = clf.predict(df_art[cols])

Here is the new DataFrame.  For example, our classifier predicts a penguin with flipper length 170 and bill length 30 is an Adelie penguin.  (Notice how there is probably no actual penguin with these measurements.)

In [ ]:
df_art

,flipper_length_mm,bill_length_mm,pred
0,170.0,30.000000,Adelie
1,170.0,30.434783,Adelie
2,170.0,30.869565,Adelie
3,170.0,31.304348,Adelie
4,170.0,31.739130,Adelie
...,...,...,...
4895,235.0,58.260870,Gentoo
4896,235.0,58.695652,Gentoo
4897,235.0,59.130435,Gentoo
4898,235.0,59.565217,Gentoo


* Make another Altair scatter plot of the predicted species, this time using `df_art`.

The most important thing to recognize from the following picture is that there are three regions (corresponding to the three classes) and that regions are separated by linear boundaries.  This partly explains why `LogisticRegression` is defined in the `linear_model` library of scikit-learn.

In [ ]:
alt.Chart(df_art).mark_circle().encode(
    x = alt.X(cols[0], scale = alt.Scale(zero=False)),
    y = alt.Y(cols[1], scale = alt.Scale(zero=False)),
    color = 'pred:N'
)

alt.Chart(...)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d93dddb6-f9a9-4f42-9ac6-4d86bae7ecb7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>